<a href="https://colab.research.google.com/github/Dhruba531/CSE225OlympicProject/blob/main/Julia_Colab_Notebook_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
using PyPlot
using Printf
using LinearAlgebra
using Flux, DiffEqFlux
using OrdinaryDiffEq
using ComponentArrays
using Random
using BSON: @save, @load
using Optimization, OptimizationOptimJL, OptimizationOptimisers
using Statistics
using Plots
gr()

# Ensure variables are defined only once
let
    # Parameters
    global ω = 1.0            # Harmonic oscillator frequency
    global X = π              # Spatial domain: [-π, π]
    global T = 2π             # Time domain: [0, 2π]
    global dx = 0.02          # Spatial step
    global dt = 0.02          # Time step
    global x = collect(-X:dx:X)
    global t = collect(0:dt:T)
    global Nx = length(x)
    global Nt = length(t)
end

# Save folder
save_folder = "tdse_data"
if isdir(save_folder)
    rm(save_folder, recursive=true)
end
mkdir(save_folder)

# Hermite polynomials for analytical solution
function hermite(n, x)
    if n == 0
        return 1.0
    elseif n == 1
        return 2x
    else
        H_prev2 = 1.0
        H_prev1 = 2x
        for i in 2:n
            H_current = 2x * H_prev1 - 2(i-1) * H_prev2
            H_prev2 = H_prev1
            H_prev1 = H_current
        end
        return H_prev1
    end
end

# Eigenstates of the harmonic oscillator
function φ_n(n, x, ω)
    norm = sqrt(sqrt(ω/π) / (2^n * factorial(n)))
    return norm * hermite(n, sqrt(ω)*x) * exp(-ω*x^2/2)
end

# Initial condition: superposition of ground and first excited states
u0 = @. (φ_n(0, x, ω) + φ_n(1, x, ω)) / sqrt(2)
v0 = zeros(Nx)
u0[1] = 0.0; u0[end] = 0.0
v0[1] = 0.0; v0[end] = 0.0

# Laplacian matrix with Dirichlet BC
lap = diagm(0 => -2.0 * ones(Nx),
            1 => ones(Nx-1),
           -1 => ones(Nx-1)) / dx^2
lap[1, :] .= 0.0
lap[1, 1] = -2.0 / dx^2
lap[1, 2] = 1.0 / dx^2
lap[end, :] .= 0.0
lap[end, end] = -2.0 / dx^2
lap[end, end-1] = 1.0 / dx^2

# TDSE RHS for true solution
function tdse_rhs!(du, u_vec, p, t)
    u = @view u_vec[1:Nx]
    v = @view u_vec[Nx+1:end]
    d2u_dx2 = lap * u
    d2v_dx2 = lap * v
    @inbounds for i in 1:Nx
        pot = 0.5 * ω^2 * x[i]^2
        du[i] = -0.5 * d2v_dx2[i] + pot * v[i]
        du[Nx+i] = 0.5 * d2u_dx2[i] - pot * u[i]
    end
    du[1] = 0.0; du[Nx] = 0.0
    du[Nx+1] = 0.0; du[2*Nx] = 0.0
end

# Solve true TDSE
prob_data = ODEProblem(tdse_rhs!, vcat(u0, v0), (0.0, T))
sol_data = solve(prob_data, Tsit5(), saveat=t, abstol=1e-10, reltol=1e-10)
ode_data = Array(sol_data)
u_data = ode_data[1:Nx, :]
v_data = ode_data[Nx+1:end, :]
prob_density = u_data.^2 + v_data.^2

# Plot true solution
fig = figure(figsize=(8,3))
sub1 = fig.add_subplot(1,2,1)
pcolormesh(x, t, prob_density', cmap="viridis")
sub1.set_xlabel("x")
sub1.set_ylabel("t")
fig.colorbar(PyPlot.gci(), ax=sub1)
sub1.set_title("True |ψ|²")

sub2 = fig.add_subplot(1,2,2)
for i in 1:10:Nt
    sub2.plot(x, prob_density[:, i], label=@sprintf("t=%.2f", t[i]))
end
sub2.set_xlabel("x")
sub2.set_ylabel("|ψ|²")
sub2.legend(fontsize=7, loc="upper left", bbox_to_anchor=(1,1))
sub2.set_title("Slices of solution")
tight_layout()
fig.savefig(joinpath(save_folder, "tdse_training_data.pdf"))

# Analytical solution for comparison
u_analytical = zeros(Nx, Nt)
v_analytical = zeros(Nx, Nt)
for j in 1:Nt
    for i in 1:Nx
        φ0 = φ_n(0, x[i], ω)
        φ1 = φ_n(1, x[i], ω)
        E0 = ω/2
        E1 = 3ω/2
        u_analytical[i,j] = (φ0 * cos(E0*t[j]) + φ1 * cos(E1*t[j])) / sqrt(2)
        v_analytical[i,j] = (-φ0 * sin(E0*t[j]) - φ1 * sin(E1*t[j])) / sqrt(2)
    end
end
u_analytical[1, :] .= 0.0; u_analytical[end, :] .= 0.0
v_analytical[1, :] .= 0.0; v_analytical[end, :] .= 0.0
prob_analytical = u_analytical.^2 + v_analytical.^2

# Animate true solution
anim = @animate for i in 1:Nt
    Plots.plot(x, prob_density[:, i],
         ylim=(0, maximum(prob_density)),
         xlabel="x", ylabel="|ψ(x,t)|²",
         title=@sprintf("TDSE at t = %.2f", t[i]),
         linewidth=3, legend=false)
end
gif_path = joinpath(save_folder, "tdse_true.gif")
gif(anim, gif_path, fps=5)